In [9]:
!pip install transformers accelerate torch sentence-transformers rouge_score --quiet

In [2]:
from google.colab import files
uploaded = files.upload()

Saving ayudas.json to ayudas.json


In [3]:
import json
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

# Directorio para guardar resúmenes
os.makedirs("resumenes_generados", exist_ok=True)

# Cargar JSON
with open("ayudas.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Función para construir prompt
def build_prompt(year_data):
    return f"""
You are an academic writer.
Write a precise and structured summary (200-250 words).
Do not invent information.

DATA:
Academic year: {year_data.get('academic_year')}
Budget: {year_data.get('total_budget_general_millions_eur')} million euros
Regulation: {year_data.get('income_thresholds_defined_by')}
Programs: {year_data.get('eligible_programs')}
Scholarship components: {year_data.get('scholarship_components')}
Academic requirements: {year_data.get('academic_requirements')}
Application period: {year_data.get('application_period')}

SUMMARY:
"""

# Función para generar resumen
def generar_resumen(model, tokenizer, prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.4,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1
    )
    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return texto.split("SUMMARY:")[-1].strip()

In [4]:
# Cargar modelo Phi-3
print("Cargando Phi-3...")
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Generar resúmenes
resumenes_phi = {}
for year_name, year_data in data.items():
    print(f"Generando resumen Phi-3 para {year_name}...")
    prompt = build_prompt(year_data)
    resumen = generar_resumen(phi_model, phi_tokenizer, prompt)
    resumenes_phi[year_name] = resumen
    # Guardar inmediatamente
    with open(f"resumenes_generados/{year_name}_phi3.txt", "w", encoding="utf-8") as f:
        f.write(resumen)

print("Resúmenes Phi-3 generados y guardados.")

# Liberar memoria GPU
del phi_model, phi_tokenizer
torch.cuda.empty_cache()

Cargando Phi-3...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Generando resumen Phi-3 para ayudas_21-22.pdf...
Generando resumen Phi-3 para ayudas_22-23.pdf...
Generando resumen Phi-3 para ayudas_23-24.pdf...
Generando resumen Phi-3 para ayudas_24-25.pdf...
Generando resumen Phi-3 para ayudas_25-26.pdf...
Resúmenes Phi-3 generados y guardados.


In [5]:
# Cargar modelo Mistral-7B
print("Cargando Mistral-7B...")
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Generar resúmenes
resumenes_mistral = {}
for year_name, year_data in data.items():
    print(f"Generando resumen Mistral-7B para {year_name}...")
    prompt = build_prompt(year_data)
    resumen = generar_resumen(mistral_model, mistral_tokenizer, prompt)
    resumenes_mistral[year_name] = resumen
    # Guardar inmediatamente
    with open(f"resumenes_generados/{year_name}_mistral.txt", "w", encoding="utf-8") as f:
        f.write(resumen)

print("Resúmenes Mistral-7B generados y guardados.")

# Liberar memoria GPU
del mistral_model, mistral_tokenizer
torch.cuda.empty_cache()

Cargando Mistral-7B...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

Generando resumen Mistral-7B para ayudas_23-24.pdf...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generando resumen Mistral-7B para ayudas_24-25.pdf...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generando resumen Mistral-7B para ayudas_25-26.pdf...
Resúmenes Mistral-7B generados y guardados.


In [15]:
# OpenLLaMA 3B

print("Cargando OpenLLaMA 3B...")

llama3b_tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b")
llama3b_model = AutoModelForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Generar resúmenes
resumenes_openllama3b = {}
for year_name, year_data in data.items():
    print(f"Generando resumen OpenLLaMA-3B para {year_name}...")
    prompt = build_prompt(year_data)
    resumen = generar_resumen(llama3b_model, llama3b_tokenizer, prompt)
    resumenes_openllama3b[year_name] = resumen

    # Guardar inmediatamente
    output_path = f"resumenes_generados/{year_name}_openllama3b.txt"
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(resumen)

# Liberar memoria GPU
del llama3b_model, llama3b_tokenizer
torch.cuda.empty_cache()

Cargando OpenLLaMA 3B...


config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/237 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Generando resumen OpenLLaMA-3B para ayudas_21-22.pdf...
Generando resumen OpenLLaMA-3B para ayudas_22-23.pdf...
Generando resumen OpenLLaMA-3B para ayudas_23-24.pdf...
Generando resumen OpenLLaMA-3B para ayudas_24-25.pdf...
Generando resumen OpenLLaMA-3B para ayudas_25-26.pdf...


In [16]:
!zip -r resumenes_generados.zip resumenes_generados
from google.colab import files
files.download("resumenes_generados.zip")

updating: resumenes_generados/ (stored 0%)
updating: resumenes_generados/ayudas_24-25.pdf_mistral.txt (deflated 50%)
updating: resumenes_generados/ayudas_25-26.pdf_mistral.txt (deflated 52%)
updating: resumenes_generados/ayudas_22-23.pdf_phi3.txt (deflated 43%)
updating: resumenes_generados/ayudas_25-26.pdf_phi3.txt (deflated 44%)
updating: resumenes_generados/ayudas_23-24.pdf_mistral.txt (deflated 49%)
updating: resumenes_generados/ayudas_24-25.pdf_phi3.txt (deflated 43%)
updating: resumenes_generados/ayudas_22-23.pdf_mistral.txt (deflated 50%)
updating: resumenes_generados/ayudas_23-24.pdf_phi3.txt (deflated 43%)
updating: resumenes_generados/ayudas_21-22.pdf_phi3.txt (deflated 43%)
updating: resumenes_generados/ayudas_21-22.pdf_mistral.txt (deflated 51%)
  adding: resumenes_generados/ayudas_23-24.pdf_openllama3b.txt (deflated 65%)
  adding: resumenes_generados/ayudas_25-26.pdf_openllama3b.txt (deflated 68%)
  adding: resumenes_generados/ayudas_21-22.pdf_openllama3b.txt (deflated 65%

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>